In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

In [14]:
recipes_clean = pd.read_csv('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/recipes.csv')
recipes_clean

,id,name,ingredients,ingredient_ids
0,137739,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,31490,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","[5481, 6324, 2499, 4717, 6276, 1170]"
2,59389,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...","[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,5289,apple a day milk shake,"['milk', 'vanilla ice cream', 'frozen apple ju...","[4717, 7474, 2946, 150]"
4,25274,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...","[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."
...,...,...,...,...
178260,185979,zydeco green beans,"['garlic cloves', 'yellow mustard seeds', 'fre...","[3203, 7978, 2725, 1336, 2794, 7803, 7655, 5369]"
178261,367912,zydeco salad,"['iceberg lettuce', 'tomatoes', '3 bean mix', ...","[4308, 7213, 13, 5007]"
178262,357451,zydeco sauce,"['mayonnaise', 'prepared horseradish', 'worces...","[4623, 3758, 7946, 7839, 7049, 5975]"
178263,188810,zydeco shrimp wrap,"['white rice', 'vegetable oil', 'onion', 'gree...","[7783, 7557, 5010, 3399, 1130, 7233, 7946, 381..."


In [16]:
ingredients_clean = pd.read_csv('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/ingredient_keys.csv')
ingredients_clean.drop(columns='Unnamed: 0',inplace=True)
ingredients_clean.head(3)

,id,replaced,count
0,4308,lettuce,4507
1,4308,lettuce,4507
2,4308,lettuce,4507


In [19]:
ingredients_clean.nunique()

id          8023
replaced    8023
count        840
dtype: int64

In [3]:
new_df = recipes_clean[['ingredient_ids']].copy()

In [4]:
new_df = new_df['ingredient_ids'].apply(lambda x: list(map(str,ast.literal_eval(x))))

In [5]:
new_df = pd.DataFrame(new_df)
new_df.head()

,ingredient_ids
0,"[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,"[5481, 6324, 2499, 4717, 6276, 1170]"
2,"[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,"[4717, 7474, 2946, 150]"
4,"[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."


In [10]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(recipes_clean['ingredient_ids'])

X.toarray();

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
result_df = pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())

In [32]:
result_df.head()

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
result_df.head(1) > 0

,10,100,1000,1001,1002,1003,1004,1005,1006,1007,...,990,991,992,993,994,995,996,997,998,999
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

A = ['very good day', 'a random thought', 'maybe like this']
B = ['so fast and slow', 'the meaning of this', 'here you go']
C = [1, 2, 3]

pdt_items = pd.DataFrame({'A':A,'B':B,'C':C})

cv = CountVectorizer()

# use pd.DataFrame here to avoid your error and add your column name    
sample = pd.DataFrame(pdt_items['A']+','+pdt_items['B']+','+pdt_items['C'].astype('str'), columns=['Output'])

In [8]:
pdt_items

,A,B,C
0,very good day,so fast and slow,1
1,a random thought,the meaning of this,2
2,maybe like this,here you go,3


In [ ]:
sample

In [ ]:
vectorized = cv.fit_transform(sample['Output'])

In [ ]:
vectorized

In [ ]:
pd.DataFrame(vectorized.toarray(),columns = cv.get_feature_names())